In [32]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from pathlib import Path
import nltk, spacy
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from collections import Counter
from gensim.parsing.preprocessing import STOPWORDS as SW

pd.options.display.float_format = '{:,.0f}'.format
in_file = Path.cwd().parents[0] / 'Processed_datasets' / 'final_db.csv'
my_stopwords = stopwords.words('english')
stopwords = set(STOPWORDS).union(my_stopwords) #preparing stopwards list
custom_stopwords = ['hi', '\n', '\n\n', '&amp;', ' ', '.', '-',
                    'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
nlp = spacy.load('en_core_web_sm', exclude=['ner', 'tok2vec', 'tagger', 'paerser', 'senter', 'lemmatizer', 'attribute_ruler']) # using only for stopwords
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

In [9]:
#1. build a random forest classifier with 'longer' text
import pandas as pd
in_file = Path.cwd() / 'Processed_datasets' / 'final_db.csv'
print(in_file)
df_train = pd.read_csv(in_file, engine='python', usecols=['Text', 'oh_label'], encoding='utf-8') #not using unnecessary columns

C:\Users\amaan\Coding\Text_minig\Project\TextMiningYAT\Processed_datasets\final_db.csv


In [54]:
from sklearn.model_selection import train_test_split

#sub_df_train = df_train[['Tokens','Label']].copy()
#label_map = dict(zip(df_train['Label'].unique().tolist(), range(6)))
#sub_df_train['Flipped_label'] = sub_df_train['Label'].apply(lambda x: label_map[x])
xs = df_train['Text'].values
ys = df_train['oh_label'].values
train_x, test_x, train_y, test_y = train_test_split(xs, ys, random_state=42, test_size=0.15, stratify=ys)
print(train_x[2005], test_x[2005])
print(type(train_x[2005]))
print(type(train_y[2005]))

"White Sox are in first place. Some of you Sox fans are so spoiled. You talk like Jim Hendry came over from the Cubs and you havent won in a hundred years. Get a grip." Ready to run playing as mkr background music 😍👌 #mkr
<class 'str'>
<class 'numpy.float64'>


In [52]:
print(test_x[2005], test_y[2005]
     
     )

Ready to run playing as mkr background music 😍👌 #mkr 0.0


In [66]:
#from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler 

PARAM_GRID = [{
    'tfidfvectorizer__stop_words' : [None, ALL_STOP_WORDS]
}]

tknzr = TweetTokenizer(preserve_case=True, reduce_len=True, strip_handles=True)
vect = TfidfVectorizer(max_df=.9, min_df=25, strip_accents='unicode', tokenizer=tknzr.tokenize)
sampler = RandomUnderSampler(random_state=42) 
clf = RandomForestClassifier()
c = make_pipeline(vect, sampler, clf)
grid_srch = GridSearchCV(
    estimator=c, param_grid=PARAM_GRID, refit=True, n_jobs=-1)
fitted = grid_srch.fit(train_x, train_y)
best_est = fitted.best_estimator_


In [8]:
vect_2 = CountVectorizer(stop_words='english', max_features=100)
c2 = make_pipeline(vect_2, clf)
c2.fit(train_x, train_y)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=100, stop_words='english')),
                ('randomforestclassifier', RandomForestClassifier())])

In [67]:
print (best_est.predict_proba(test_x[0:10]))#, c2.predict_proba(test_x[0:10]))

[[0.37       0.63      ]
 [0.43       0.57      ]
 [0.71       0.29      ]
 [0.88775134 0.11224866]
 [0.75454545 0.24545455]
 [0.95       0.05      ]
 [0.66376263 0.33623737]
 [0.76       0.24      ]
 [0.66       0.34      ]
 [0.53       0.47      ]]


In [57]:
best_clf = best_est[-1]
print(best_clf)

RandomForestClassifier()


In [60]:
print(best_est)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=100,
                                 stop_words={'\n', '\n\n', ' ', '$', '&amp;',
                                             "'d", "'ll", "'m", "'re", "'s",
                                             "'ve", '-', '.', '@', 'a', 'about',
                                             'above', 'across', 'after',
                                             'afterwards', 'again', 'against',
                                             'ain', 'all', 'almost', 'alone',
                                             'along', 'already', 'also',
                                             'although', ...})),
                ('randomundersampler', RandomUnderSampler(random_state=42)),
                ('randomforestclassifier', RandomForestClassifier())])


In [63]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer()

In [64]:
idx = 21
exp = explainer.explain_instance(test_x[idx], best_est.predict_proba, num_features=6, )
print('Document id: %d' % idx)
print('Probability(non-toxic) =', best_est.predict_proba([test_x[idx]])[0, 1])
print('True class: %s' % class_names[int(test_y[idx])])
print('Tweet: %s' % test_x[idx])

Document id: 21
Probability(non-toxic) = 1.0
True class: Toxic
Tweet: @brendlewhat @MaxBlumenthal @veganforareason @dhere Christians and Hindus murdered for blasphemy by Muslim mobs. http://t.co/f3aWjksaUz


In [30]:
print(test_x[500
            ])

"Haley, all of your interviews are great, because you are great!\nand wonderful! and beautiful! and ... I love you."


In [29]:
exp_2 = explainer.explain_instance(test_x[idx], c.predict_proba, num_features=6)
print(exp.as_list(), exp_2.as_list())


[('hate', 0.15131205521371688), ('shadowrun', -0.05346704258417674)] [('shadowrun', 1.1859480308508826e-30), ('hate', 1.1745284392340069e-30)]
